### Evaluation

In [24]:
def load_result(filename,ori_file):
    file =open(filename,'r',encoding='UTF-8')    
    label=[]
    pre=[]
    for i in file:
        words = i.split()
        if len(words)>2:
            #token.append(words[0])
            label.append(words[1])
            pre.append(words[2])
    file.close()
    
    ori_file =open(ori_file,'r',encoding='UTF-8')
    token=[]
    for i in ori_file:
        words = i.split()
        
        if len(words)>1:
            token.append(words[0])
    ori_file.close()
    
    return token,label,pre


In [2]:
def token_precision_and_recall(label,pre):
    TP=0
    FP=0
    FN=0
    for i in range(len(label)):
        if label[i]!="O" and pre[i]!="O":
            if label[i][2:]==pre[i][2:]:
                TP+=1
            else:
                FP+=1
        
        elif label[i]=="O" and pre[i]!="O":
            FP+=1
        elif label[i]!="O" and pre[i]=="O":
            FN+=1
        
    precision = TP/(TP+FP+0.1)
    recall = TP/(TP+FN+0.1)
    return precision,recall

    

In [3]:

def specific_category(label,spec):
    cleaned = []
    for i in label:
        if i =="O":
            cleaned.append(i)
        else:
            if i[2:]==spec:
                cleaned.append(i)
            else:
                cleaned.append("O")
    return cleaned
        

def token_precision_and_recall_specific(label,pre,spec):
    label=specific_category(label,spec)
    pre = specific_category(pre,spec)
    
    num_label = len(label)-label.count("O")
    num_pre = len(pre)-pre.count("O")
    
    TP=0
    FP=0
    FN=0
    for i in range(len(label)):
        if label[i]!="O" and pre[i]!="O":
            TP+=1       
        elif label[i]=="O" and pre[i]!="O":
            FP+=1
        elif label[i]!="O" and pre[i]=="O":
            FN+=1
        
    precision = TP/(TP+FP+0.1)
    recall = TP/(TP+FN+0.1)
    return precision,recall,num_label,num_pre

In [4]:
def phrase_recall(token,label,pre):
    token_ph=[]
    label_ph=[]
    pre_ph=[]

    label_temp = ""
    token_ph_list = []
    pre_ph_list = []

    for i in range(len(label)):
        if label[i][0]=="B":
            label_ph.append(label_temp)
            token_ph.append(token_ph_list)
            pre_ph.append(pre_ph_list)
        
            label_temp = ""
            token_ph_list = []
            pre_ph_list = []
        
            label_temp = label[i][2:]
            token_ph_list.append(token[i])
            pre_ph_list.append(pre[i][2:])
        
        elif label[i][0]=="I":
            token_ph_list.append(token[i])
            pre_ph_list.append(pre[i][2:])
            
    label_ph.append(label_temp)
    token_ph.append(token_ph_list)
    pre_ph.append(pre_ph_list)
    
    label_ph.pop(0)
    token_ph.pop(0)
    pre_ph.pop(0)
    
    TP=0
    FN=0
    for i in range(len(label_ph)):
        if label_ph[i] in pre_ph[i]:
            TP+=1
        else:
            FN+=1

    recall = TP/(TP+FN+0.1)      
    
    return token_ph, label_ph, pre_ph, recall



In [5]:
def phrase_recall_specific(token_ph, label_ph, pre_ph, specific):
    TP=0
    FN=0
    for i in range(len(label_ph)):
        if label_ph[i]==specific:
            if label_ph[i] in pre_ph[i]:
                TP+=1
            else:
                FN+=1

    recall = TP/(TP+FN+0.1)      
    
    return recall


In [6]:
def phrase_precision(token,label,pre):
    token_ph=[]
    label_ph=[]
    pre_ph=[]

    pre_temp = ""
    token_ph_list = []
    label_ph_list = []
    
    start = True

    for i,p in enumerate(pre):
        if len(p)>2:
            if p.startswith("B"):
                start = True
            if start:
            
                pre_ph.append(pre_temp)
                token_ph.append(token_ph_list)
                label_ph.append(label_ph_list)

                pre_temp = ""
                token_ph_list = []
                label_ph_list = []

                pre_temp = p[2:]
                token_ph_list.append(token[i])
                label_ph_list.append(label[i][2:])

            else:
                token_ph_list.append(token[i])
                label_ph_list.append(label[i][2:])
                
            start = False
        
        else:
            start = True
            
    pre_ph.append(pre_temp)
    token_ph.append(token_ph_list)
    label_ph.append(label_ph_list)
    
    label_ph.pop(0)
    token_ph.pop(0)
    pre_ph.pop(0)
    
    TP=0
    FP=0
    for i in range(len(pre_ph)):
        if pre_ph[i] in label_ph[i]:
            TP+=1
        else:
            FP+=1

    precision = TP/(TP+FP+0.1)      
    
    return token_ph, label_ph, pre_ph, precision




In [7]:
def phrase_precision_specific(token_ph, label_ph, pre_ph, specific):
    TP=0
    FP=0
    for i in range(len(pre_ph)):
        if pre_ph[i]==specific:
            if pre_ph[i] in label_ph[i]:
                TP+=1
            else:
                FP+=1

    precision = TP/(TP+FP+0.1)      
    
    return precision



In [25]:
result_path = "../bert_ner/temp/label_test.txt"
ori_file_path = "../data/test/test_data/test_single_layer.txt"

token,label,pre = load_result(result_path, ori_file_path)
precision,recall = token_precision_and_recall(label,pre)
F1_token = (2*precision*recall)/(precision+recall+0.1)

token_ph_r, label_ph_r, pre_ph_r, recall_ph = phrase_recall(token,label,pre)
token_ph_p, label_ph_p, pre_ph_p, precision_ph = phrase_precision(token,label,pre)
F1_ph = (2*precision_ph*recall_ph)/(precision_ph+recall_ph+0.1)
#acc_ph = accuracy_ph(label_ph,pre_ph)
#precision_ph,recall_ph = precision_and_recall_ph(label_ph,pre_ph)
print("over_all")
print("Token-level:")
print("count: ",len(label)-label.count("O"))
print("predicted: ",len(pre)-pre.count("O"))
print("Precision: ",precision)
print("Recall: ",recall)
print("F1: ",F1_token)
print()
print("Phase-level:")
print("count",len(label_ph_r))
print("predicted",len(pre_ph_p))
print("Precision: ",precision_ph)
print("Recall: ",recall_ph)
print("F1: ",F1_ph)

over_all
Token-level:
count:  668
predicted:  497
Precision:  0.7262120297726815
Recall:  0.6035779969904698
F1:  0.6131328294587249

Phase-level:
count 353
predicted 355
Precision:  0.7096592509152351
Recall:  0.7080147267063155
F1:  0.6621306130303174


In [26]:
import pandas as pd

In [27]:
df_dic = {"category":[],
          "token_count":[],
          "token_detected":[],
          "token_precision":[],
          "token_recall":[],
          "token_F1":[],
         "phrase_count":[],
          "phrase_detected":[],
         "phrase_precision":[],
         "phrase_recall":[],
          "phrase_F1":[]
         }
for cate in set(label_ph_r):

    count_phrase=label_ph_r.count(cate)
    detected_phrase=pre_ph_p.count(cate)
    precision_phrase=phrase_precision_specific(token_ph_p, label_ph_p, pre_ph_p, cate)
    recall_phrase=phrase_recall_specific(token_ph_r, label_ph_r, pre_ph_r, cate)
    F1_phrase = (2*precision_phrase*recall_phrase)/(precision_phrase+recall_phrase+0.1)
    
    token_prec,token_rec,token_count,token_num_pre = token_precision_and_recall_specific(label,pre,cate)
    token_F1 = (2*token_prec*token_rec)/(token_prec+token_rec+0.1)
    
    df_dic["category"].append(cate)
    df_dic["token_count"].append(token_count)
    df_dic["token_detected"].append(token_num_pre)
    df_dic["token_precision"].append(token_prec)
    df_dic["token_recall"].append(token_rec)
    df_dic["token_F1"].append(token_F1)
    
    df_dic["phrase_count"].append(count_phrase)
    df_dic["phrase_detected"].append(detected_phrase)
    df_dic["phrase_precision"].append(precision_phrase)
    df_dic["phrase_recall"].append(recall_phrase)
    df_dic["phrase_F1"].append(F1_phrase)
    


In [28]:
pd.DataFrame(df_dic)

,category,token_count,token_detected,token_precision,token_recall,token_F1,phrase_count,phrase_detected,phrase_precision,phrase_recall,phrase_F1
0,Amount,54,51,0.724070,0.683919,0.656776,24,32,0.591900,0.746888,0.614522
1,Type,48,30,0.730897,0.457380,0.518984,29,28,0.747331,0.756014,0.704767
2,MedicalCondition,5,0,0.000000,0.000000,0.000000,4,0,0.000000,0.000000,0.000000
3,MaritalStatus,21,15,0.927152,0.663507,0.727730,13,15,0.927152,0.992366,0.911182
4,ExposureHistory,23,14,0.780142,0.476190,0.547795,6,8,0.740741,0.983607,0.798749
5,Method,9,2,0.476190,0.109890,0.152544,8,2,0.476190,0.123457,0.168053
6,Extent,4,0,0.000000,0.000000,0.000000,2,0,0.000000,0.000000,0.000000
7,Occupation,33,29,0.446735,0.392749,0.373513,15,25,0.438247,0.794702,0.522564
8,PhysicalActivity,22,0,0.000000,0.000000,0.000000,9,0,0.000000,0.000000,0.000000
9,Family,29,22,0.633484,0.481100,0.501849,19,16,0.496894,0.366492,0.378058


### Error

In [43]:
missed_dic = {"ori_phrase":[],
             "ture_label":[],
             "predict_labels":[]}

for i in range(len(label_ph_r)):
    if label_ph_r[i] not in pre_ph_r[i]:
        missed_dic["ori_phrase"].append(token_ph_r[i])
        missed_dic["ture_label"].append(label_ph_r[i])
        missed_dic["predict_labels"].append(["O" if i=="" else i for i in pre_ph_r[i]])

missed_df = pd.DataFrame(missed_dic)   
missed_df

,ori_phrase,ture_label,predict_labels
0,"[He, has]",Status,"[O, O]"
1,[daughter],Family,[O]
2,"[history, of]",Status,"[O, O]"
3,[user],Status,[O]
4,[lives],LivingSituation,[O]
...,...,...,...
98,[admits],Status,[O]
99,[exercising],PhysicalActivity,[Type]
100,"[exercise, patterns]",PhysicalActivity,"[O, O]"
101,"[within, normal, limits]",Amount,"[O, O, O]"


In [42]:
wrong_dic = {"ori_phrase":[],
             "ture_labels":[],
             "predict_label":[]}

for i in range(len(pre_ph_p)):
    if pre_ph_p[i] not in label_ph_p[i]:
        wrong_dic["ori_phrase"].append(token_ph_p[i])
        wrong_dic["predict_label"].append(pre_ph_p[i])
        wrong_dic["ture_labels"].append(["O" if i=="" else i for i in label_ph_p[i]])

wrong_df = pd.DataFrame(wrong_dic)
wrong_df

,ori_phrase,ture_labels,predict_label
0,[alone],[Method],LivingSituation
1,[social],[O],Amount
2,"[laboratory, technician]","[Occupation, Occupation]",Type
3,[Cornell],[Location],Occupation
4,[Laboratory],[Location],Occupation
...,...,...,...
98,[banker],[Type],Occupation
99,"[does, not]","[O, O]",Status
100,[no],[O],Status
101,[kids],[O],Family
